In [ ]:
#Programme contenant le centre de rotation

# Importer les bibliothèques 
import tomopy
import dxchange
import matplotlib.pyplot as plt
import datetime
import time
from imagestacks import create_animation
from IPython.core.display import HTML

#Definir le fichiers contenants les images de tomographie
fname = '/Users/leapommier/Desktop/projet multi/tilt_0_180/carteSD120321_0000.tif'
ind = [i for i in range(180)] 
print(ind)
print(fname)

start = datetime.datetime.now()
proj = dxchange.reader.read_tiff_stack(fname, ind)
end = datetime.datetime.now() 
duree = end - start #Pour mesurer le temps mis pour effectuer la ligne proj
print(duree)


#diminuer la taille des images pour diminuer le temps d'execution
print('avant',proj.shape)
down = tomopy.misc.morph.downsample(proj, level=1, axis=1)#axe 1 lignes: faire la moyenne des pixels 2 à 2
down = tomopy.misc.morph.downsample(down, level=1, axis=2)# axe 2 colonnes: faire la moyenne des pixels 2 à 2
#Avec axe 0 on aurait pris l'image complète qu'on aurait moyener avec tous les tilts ce qui aurait eu un rendu flou
print('apres',down.shape)


#afficher la  1ere image avant downsampling
image0 = proj[0, :, :]#image zéro, on prend tous les pixels horizontaux et verticaux: toute la matrice des pixels
plt.imshow(image0, cmap='Greys_r')
plt.show()


#afficher la  1ere image apres downsampling
image0 = down[0, :, :]
plt.imshow(image0, cmap='Greys_r')
plt.show()

#Animation permettant de voir toute les photos 
anim2 = create_animation(down, size=4)
display(HTML(anim2.to_jshtml()))


#normaliser avec loi log pour loi de beer lambert
norm = tomopy.prep.normalize.minus_log(down, ncore=None, out=None)


#afficher la  1ere image normalisee
image0 = norm[0, :, :]
plt.imshow(image0, cmap='Greys_r')
plt.show()


ang = tomopy.angles(180) # Genere des angles de 0° à 180°


#anim2 = create_animation(norm, size=4)
#display(HTML(anim2.to_jshtml()))

#definition du centre de rotation et
rot_center = tomopy.find_center(norm, ang, init=None, ind=0, tol=0.5)
start = datetime.datetime.now()
rec = tomopy.recon(norm, ang, center=rot_center, algorithm='gridrec') # Reconstruction objet avec prise en compte du centre de rotation.

#Afficher l'image 25
plt.imshow(rec[256,:,:])
plt.show()


#anim2 = create_animation(rec, size=4)
#display(HTML(anim2.to_jshtml()))
#end = datetime.datetime.now()
#duree = end - start
#duree
#print(duree)